# Introduction

Python의 Model I/O에 대해 학습함

## FewShotPromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [4]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [5]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'question': 'What do you know about Italy?', 'answer': '\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'question': 'What do you know about Greece?', 'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}'), suffix='Human: What do you know about {country}?')

In [6]:

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella and Tapas
        Currency: Euro

AIMessage(content='AI: I know this:\n        Capital: Madrid\n        Language: Spanish\n        Food: Paella and Tapas\n        Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--c62e0a9d-6cc5-4cbb-bea1-bdc1931fd1de-0')

## FewShotChatMessagePromptTemplate

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [8]:
example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}"),
  ]
)

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you kjnow about {country}?")
  ]
)

In [9]:
chain= final_prompt | chat

chain.invoke(
  {
    "country" : "Korea"
  }
)


        I know this:
        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).
        Languages: Korean
        Food: Kimchi and Bibimbap
        Currency: North Korean Won (North) and South Korean Won (South)

AIMessage(content='\n        I know this:\n        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).\n        Languages: Korean\n        Food: Kimchi and Bibimbap\n        Currency: North Korean Won (North) and South Korean Won (South)', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--f6b328e7-149c-468f-b945-3b0c0e293d8d-0')

## LengthBasedExampleSelector

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [11]:
class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples
  
  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice

    return [choice(self.examples)]

In [12]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
  examples=examples
)

In [13]:
prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## Serialization and Composition

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

In [15]:
intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
  """
)

example = PromptTemplate.from_template(
  """
  this is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
  """
)

start= PromptTemplate.from_template(
  """
  Strat now!

  Human: {question}
  You:
  """
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
  """
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

/var/folders/jw/cgbm__d937x01nmp9bx6453r0000gn/T/ipykernel_24392/423968787.py:42: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  full_prompt = PipelinePromptTemplate(


Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--e10b53c6-c85e-423e-90c9-579f6d1f85f7-0')